In [2]:
import os
import math
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd

print("TensorFlow:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

# ------------------------------------------------
# 1) تحميل بيانات Fashion-MNIST وتحضيرها
# ------------------------------------------------
from tensorflow.keras.datasets import fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# تطبيع الصور إلى [0,1]
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

# تحضير بيانات الـ MLP (Flatten لاحقًا في النموذج)
x_train_mlp = x_train  # (N, 28, 28)
x_test_mlp  = x_test

# تحضير بيانات الـ CNN بإضافة بُعد القناة: (N,28,28,1)
x_train_cnn = np.expand_dims(x_train, axis=-1)  # (N, 28, 28, 1)
x_test_cnn  = np.expand_dims(x_test, axis=-1)

print("Shapes:")
print(" - x_train_mlp:", x_train_mlp.shape)
print(" - x_train_cnn:", x_train_cnn.shape)

TensorFlow: 2.19.0
GPU available: []
29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Shapes:
 - x_train_mlp: (60000, 28, 28)
 - x_train_cnn: (60000, 28, 28, 1)


In [3]:
# ------------------------------------------------
# 2) بناء النماذج
# ------------------------------------------------
# 2.1) نموذج الـ MLP
mlp_model = keras.Sequential(
    [
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(256, activation="relu"),
        layers.Dense(128, activation="relu"),
        layers.Dense(10, activation="softmax"),
    ],
    name="MLP_Model",
)
mlp_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
print("\n=== MLP model summary ===")
mlp_model.summary()

# 2.2) نموذج الـ CNN
cnn_model = keras.Sequential(
    [
        layers.Conv2D(16, kernel_size=3, activation="relu", input_shape=(28, 28, 1)),
        layers.MaxPooling2D(pool_size=2),
        layers.Conv2D(32, kernel_size=3, activation="relu"),
        layers.MaxPooling2D(pool_size=2),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(10, activation="softmax"),
    ],
    name="CNN_Model",
)
cnn_model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
print("\n=== CNN model summary ===")
cnn_model.summary()



=== MLP model summary ===


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "MLP_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       200,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 235,146 (918.54 KB)

...

Model: "CNN_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 800)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,714 (221.54 KB)

 Trainable params: 56,714 (221.54 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# ------------------------------------------------
# 3) التدريب والتقييم
# ------------------------------------------------
EPOCHS = 5
BATCH  = 64

print("\n=== Training MLP ===")
hist_mlp = mlp_model.fit(
    x_train_mlp, y_train,
    epochs=EPOCHS,
    batch_size=BATCH,
    validation_split=0.1,
    verbose=2
)

print("\n=== Training CNN ===")
hist_cnn = cnn_model.fit(
    x_train_cnn, y_train,
    epochs=EPOCHS,
    batch_size=BATCH,
    validation_split=0.1,
    verbose=2
)

print("\n=== Evaluate on test set ===")
test_loss_mlp, test_acc_mlp = mlp_model.evaluate(x_test_mlp, y_test, verbose=0)
test_loss_cnn, test_acc_cnn = cnn_model.evaluate(x_test_cnn, y_test, verbose=0)

print(f"MLP - Test Accuracy: {test_acc_mlp:.4f} | Test Loss: {test_loss_mlp:.4f}")
print(f"CNN - Test Accuracy: {test_acc_cnn:.4f} | Test Loss: {test_loss_cnn:.4f}")



=== Training MLP ===
Epoch 1/5
844/844 - 5s - 5ms/step - accuracy: 0.8223 - loss: 0.5022 - val_accuracy: 0.8630 - val_loss: 0.3942
Epoch 2/5
844/844 - 3s - 4ms/step - accuracy: 0.8656 - loss: 0.3698 - val_accuracy: 0.8582 - val_loss: 0.3882
Epoch 3/5
844/844 - 4s - 5ms/step - accuracy: 0.8795 - loss: 0.3292 - val_accuracy: 0.8740 - val_loss: 0.3561
Epoch 4/5
844/844 - 3s - 4ms/step - accuracy: 0.8893 - loss: 0.3025 - val_accuracy: 0.8690 - val_loss: 0.3609
Epoch 5/5
844/844 - 3s - 4ms/step - accuracy: 0.8940 - loss: 0.2866 - val_accuracy: 0.8850 - val_loss: 0.3239

=== Training CNN ===
Epoch 1/5
844/844 - 15s - 18ms/step - accuracy: 0.7860 - loss: 0.5942 - val_accuracy: 0.8455 - val_loss: 0.4300
Epoch 2/5
844/844 - 14s - 16ms/step - accuracy: 0.8604 - loss: 0.3935 - val_accuracy: 0.8700 - val_loss: 0.3776
Epoch 3/5
844/844 - 14s - 17ms/step - accuracy: 0.8779 - loss: 0.3437 - val_accuracy: 0.8795 - val_loss: 0.3427
Epoch 4/5
844/844 - 15s - 17ms/step - accuracy: 0.8872 - loss: 0.3113 

In [5]:
# === Q4: TFLite Full-Integer Quantization ===
import os, numpy as np, tensorflow as tf
os.makedirs("models", exist_ok=True)

# حفظ بصيغة Keras H5 (اختياري لكنه مفيد للمقارنة)
mlp_h5_path = "models/mlp_model.h5"
cnn_h5_path = "models/cnn_model.h5"
mlp_model.save(mlp_h5_path)
cnn_model.save(cnn_h5_path)

# ممثل البيانات (عدّل التطبيع حسب بياناتك)
def rep_dataset_mlp(num_batches=100, batch_size=1):
    idx = np.random.choice(len(x_train_mlp), size=min(len(x_train_mlp), num_batches*batch_size), replace=False)
    for i in range(0, len(idx), batch_size):
        yield [x_train_mlp[idx[i:i+batch_size]].astype(np.float32)]

def rep_dataset_cnn(num_batches=100, batch_size=1):
    idx = np.random.choice(len(x_train_cnn), size=min(len(x_train_cnn), num_batches*batch_size), replace=False)
    for i in range(0, len(idx), batch_size):
        yield [x_train_cnn[idx[i:i+batch_size]].astype(np.float32)]

# تحويل وكمّنة MLP
converter_mlp = tf.lite.TFLiteConverter.from_keras_model(mlp_model)
converter_mlp.optimizations = [tf.lite.Optimize.DEFAULT]
converter_mlp.representative_dataset = rep_dataset_mlp
converter_mlp.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter_mlp.inference_input_type = tf.int8
converter_mlp.inference_output_type = tf.int8
tflite_mlp = converter_mlp.convert()
mlp_tflite_path = "models/mlp_int8.tflite"
with open(mlp_tflite_path, "wb") as f: f.write(tflite_mlp)

# تحويل وكمّنة CNN
converter_cnn = tf.lite.TFLiteConverter.from_keras_model(cnn_model)
converter_cnn.optimizations = [tf.lite.Optimize.DEFAULT]
converter_cnn.representative_dataset = rep_dataset_cnn
converter_cnn.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter_cnn.inference_input_type = tf.int8
converter_cnn.inference_output_type = tf.int8
tflite_cnn = converter_cnn.convert()
cnn_tflite_path = "models/cnn_int8.tflite"
with open(cnn_tflite_path, "wb") as f: f.write(tflite_cnn)

print("Saved:", mlp_tflite_path, "and", cnn_tflite_path)


Saved artifact at '/tmp/tmpyi42bgj3'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  134430280578064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134430280579024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134430280578448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134430280578832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134430280579216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134430280577104: TensorSpec(shape=(), dtype=tf.resource, name=None)


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Saved artifact at '/tmp/tmpz2ia58pz'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='keras_tensor_5')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  134430233772496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134430233773264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134430280580752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134430233773456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134430233773840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134430233775184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134430233776144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134430233775376: TensorSpec(shape=(), dtype=tf.resource, name=None)
Saved: models/mlp_int8.tflite and models/cnn_int8.tflite


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [6]:
# === Q5: Size Comparison Table ===
import os

def file_mb(path): return os.path.getsize(path)/(1024*1024)

keras_mlp_mb = file_mb(mlp_h5_path)
keras_cnn_mb = file_mb(cnn_h5_path)
tflite_mlp_mb = file_mb(mlp_tflite_path)
tflite_cnn_mb = file_mb(cnn_tflite_path)

SRAM_MB = 0.5  # 512KB
def sram_ok(flt_mb):
    SRAM_MB = 0.5  # 512KB
    return flt_mb <= SRAM_MB


print("\n=== Model Size Comparison (MB) ===")
print(f"{'Model':<8} | {'Keras':>8} | {'TFLite INT8':>12} | {'SRAM512KB OK?':>13}")
print("-"*55)
print(f"{'MLP':<8} | {keras_mlp_mb:>8.4f} | {tflite_mlp_mb:>12.4f} | {str(sram_ok(tflite_mlp_mb)):>13}")
print(f"{'CNN':<8} | {keras_cnn_mb:>8.4f} | {tflite_cnn_mb:>12.4f} | {str(sram_ok(tflite_cnn_mb)):>13}")



=== Model Size Comparison (MB) ===
Model    |    Keras |  TFLite INT8 | SRAM512KB OK?
-------------------------------------------------------
MLP      |   2.7232 |       0.2370 |          True
CNN      |   0.6874 |       0.0610 |          True


In [7]:
# === Q6: Quick TFLite Accuracy Check ===
import numpy as np, tensorflow as tf

def tflite_accuracy(tflite_path, x_test, y_test, max_samples=500):
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()
    input_details  = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    in_scale, in_zero = input_details[0]['quantization']

    n = min(len(x_test), max_samples)
    correct = 0
    for i in range(n):
        x = x_test[i:i+1].astype(np.float32)
        x_q = np.round(x / in_scale + in_zero).astype(np.int8)
        interpreter.set_tensor(input_details[0]['index'], x_q)
        interpreter.invoke()
        out = interpreter.get_tensor(output_details[0]['index'])
        pred = np.argmax(out, axis=-1)[0]
        if pred == y_test[i]:
            correct += 1
    return correct/n

try:
    acc_tfl_mlp = tflite_accuracy(mlp_tflite_path, x_test_mlp, y_test, max_samples=500)
    acc_tfl_cnn = tflite_accuracy(cnn_tflite_path, x_test_cnn, y_test, max_samples=500)
    print(f"TFLite INT8 Accuracies (subset) -> MLP: {acc_tfl_mlp:.4f} | CNN: {acc_tfl_cnn:.4f}")
except Exception as e:
    print("(Skipped quick TFLite eval)", e)


TFLite INT8 Accuracies (subset) -> MLP: 0.8840 | CNN: 0.9000


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [8]:
# === Q7: Deployment Feasibility Summary (XIAO ESP32S3) ===
print("\n=== Deployment Feasibility on XIAO ESP32S3 ===")
print("- SRAM ~ 512KB؛ يجب أن يكون نموذج TFLite INT8 صغيرًا جدًا + معاملات وسيطة قليلة.")
print("- إن كان عمود 'SRAM512KB OK?' True أعلاه فهو مؤشر مبدئي جيد، لكن القياس الحقيقي على اللوحة ضروري.")
print("- تردّد حتى ~240MHz؛ نماذج صغيرة INT8 قد تعطي زمن استدلال < 100ms اعتمادًا على أبعاد الإدخال والطبقات.")
print("- استخدم MicroTensor Arena بحجم مناسب وجرب تقليله تدريجيًا حتى يستقر التنفيذ دون Out-Of-Memory.")



=== Deployment Feasibility on XIAO ESP32S3 ===
- SRAM ~ 512KB؛ يجب أن يكون نموذج TFLite INT8 صغيرًا جدًا + معاملات وسيطة قليلة.
- إن كان عمود 'SRAM512KB OK?' True أعلاه فهو مؤشر مبدئي جيد، لكن القياس الحقيقي على اللوحة ضروري.
- تردّد حتى ~240MHz؛ نماذج صغيرة INT8 قد تعطي زمن استدلال < 100ms اعتمادًا على أبعاد الإدخال والطبقات.
- استخدم MicroTensor Arena بحجم مناسب وجرب تقليله تدريجيًا حتى يستقر التنفيذ دون Out-Of-Memory.
